# To-mel
Script to convert an input (GTZAN dataset) to mel spectrogram for training data.
Mel spectograms logarithmically render frequency above a given threshold.

In [ ]:
# Libraries
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Get file path to audio file
path = os.path.abspath("C:/Users/uqnpipe2/Projects/genre-match/dev/disco.00000.au")

# Load audio file as python audio object
y, sr = librosa.load(path)

# Define spectrogram dimensions
fig, ax = plt.subplots()
height = 128 # Height of image

# Generate a mel spectrogram array
S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=height, fmax=8000)

# Generate spectrogram
S_dB = librosa.power_to_db(S, ref=np.max)
img = librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000, ax=ax)

# Generate filename
name = os.path.basename(path)

# Generate savepath
genre, track, filetype = name.split(".")
savepath = "../dev/" + genre + "/" + genre + "." + track + ".png"

# Check savepath exists, mkdir otherwise
if os.path.exists(savepath) is False:
    os.mkdir("../dev/" + genre + "/")

# Write spectrogram to file
plt.savefig(savepath)
plt.show()